In [1]:
version = "REPLACE_PACKAGE_VERSION"

---
# Assignment 1 Part 3: Hidden Markov Models (HMMs) (30 pts)

In this assignment, we're going to train a Hidden Markov Model (HMM) that is able to tag words with their [part-of-speech](https://en.wikipedia.org/wiki/Part_of_speech) (POS) in a sentence. 

In [1]:
# Configure nltk

import nltk

nltk_data_path = "assets/nltk_data"
if nltk_data_path not in nltk.data.path:
    nltk.data.path.append(nltk_data_path)

## Question 1: Load the dataset (20 pts)

Let's first load some POS-tagged data for training and testing our HMM. **Complete the `load_data` function below** that takes two arguments, `data_file`, which specifies the data file to read, and `label`, which indicates whether to include labels in your data.

Under the `assets/conll03` folder, you are given three data files, `eng.train`, `eng.testa` and `eng.testb`, that come from a [CoNLL-2003 shared task](https://www.clips.uantwerpen.be/conll2003/ner/). The shared task was originally held as a competition for Named Entity Recognition (NER), but the data it provided also includes POS tags that make POS Tagging possible. NER and POS Tagging are very similar tasks and HMMs are capable of handling both of them. After finishing this assignment, you are encouraged to re-purpose your HMM tagger for NER as a self-exercise. 

All three data files share the same format. They are concatenation of several documents demarcated by `-DOCSTART- -X- O O`. Sentences are separated by empty lines and each token occupies one line with its POS tag. For example, 

`scientists NNS I-NP O`

says the token `scientists` has a POS tag of `NNS` (noun, plural). If interested, see [here](https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html) for a reference list of all POS tags. The [CoNLL-2003 shared task](https://www.clips.uantwerpen.be/conll2003/ner/) page also includes some descriptions of the data. 

In many NLP tasks, it's commonplace to divide a dataset into training, development and testing sets. Since training and development sets are used for model learning and hyper-parameter tuning, they often include labels (if any). However, labels are often *not* distributed with testing sets to prevent anyone from cheating, especially in competitions. Here we will follow the same convention, by taking `eng.train` as the training set, `eng.testa` as the development set and `eng.testb` as the testing set, and **will only allow the training and development sets to include labels** by specifying the `label` argument: 

* **When called with `label=True`**, your function should return a `list`, where each element is a `list` of `tuple` representing each sentence. Each `tuple` contains a token and its POS tag, such as `("scientists", "NNS")`. 

* **When called with `label=False`**, your function should also return a `list`, where each element is just a `list` of tokens. POS tags are not included and `tuple` is not needed.

Finally, **in all the three data files, we only consider "substantial" sentences that have at least $10$ tokens**. You'd need to filter out shorter sentences and should not include them in any of your three subsets of the data. 

**This function should return a `list`, where each element is either a `list` of `tuple` or a `list` of `str`.**

**When the argument `label=True`, an example output would be:**
```
[[('The', 'DT'), ('European', 'NNP'), ('Commission', 'NNP'), ('said', 'VBD'), ('on', 'IN'), ('Thursday', 'NNP'), ('it', 'PRP'), ('disagreed', 'VBD'), ('with', 'IN'), ('German', 'JJ'), ('advice', 'NN'), ('to', 'TO'), ('consumers', 'NNS'), ('to', 'TO'), ('shun', 'VB'), ('British', 'JJ'), ('lamb', 'NN'), ('until', 'IN'), ('scientists', 'NNS'), ('determine', 'VBP'), ('whether', 'IN'), ('mad', 'JJ'), ('cow', 'NN'), ('disease', 'NN'), ('can', 'MD'), ('be', 'VB'), ('transmitted', 'VBN'), ('to', 'TO'), ('sheep', 'NN'), ('.', '.')], 
...
[('Africa', 'NNP'), (')', ')'), ('71', 'CD'), ('74', 'CD'), ('75', 'CD'), (',', ','), ('David', 'NNP'), ('Gilford', 'NNP'), ('69', 'CD'), ('74', 'CD'), ('77', 'CD'), ('.', '.')]]
```

**When the argument `label=False`, an example output would be:**

```
[['SOCCER', '-', 'JAPAN', 'GET', 'LUCKY', 'WIN', ',', 'CHINA', 'IN', 'SURPRISE', 'DEFEAT', '.'], 
 ...
 ['The', 'lanky', 'former', 'Leeds', 'United', 'defender', 'did', 'not', 'make', 'his', 'England', 'debut', 'until', 'the', 'age', 'of', '30', 'but', 'eventually', 'won', '35', 'caps', 'and', 'was', 'a', 'key', 'member', 'of', 'the', '1966', 'World', 'Cup', 'winning', 'team', 'with', 'his', 'younger', 'brother', ',', 'Bobby', '.']]
```

In [60]:
def load_data(data_file, label=True):
    """
    Load tokens (and labels, if allowed) from a data_file
    """
    text = open('assets/conll03/'+data_file,'r').read().split("\n\n")

    all_sentences = []
    for s in text:
        all_sentences.append(s.split('\n'))


    sentences = []    
    for t in all_sentences:
        if len(t) > 9:
                 sentences.append(t)

    tagged_sents = []
    for s in sentences:
        sent = []
        for t in s:
            if label == True:
                u = t.split()[:2]
                sent.append(tuple(u))
            else:
                u = t.split()[0]
                sent.append(u)
        tagged_sents.append(sent)
    
    # YOUR CODE HERE
    #raise NotImplementedError()
    
    return tagged_sents

In [59]:
data_file = 'eng.train' 

text = open('assets/conll03/'+data_file,'r').read().split("\n\n")
label = False

all_sentences = []
for s in text:
    all_sentences.append(s.split('\n'))
    
    
sentences = []    
for t in all_sentences:
    if len(t) > 9:
             sentences.append(t)

tagged_sents = []
for s in sentences:
    sent = []
    for t in s:
        if label == True:
            u = t.split()[:2]
            sent.append(tuple(u))
        else:
            u = t.split()[0]
            sent.append(u)
    tagged_sents.append(sent)
            
tagged_sents





[['The',
  'European',
  'Commission',
  'said',
  'on',
  'Thursday',
  'it',
  'disagreed',
  'with',
  'German',
  'advice',
  'to',
  'consumers',
  'to',
  'shun',
  'British',
  'lamb',
  'until',
  'scientists',
  'determine',
  'whether',
  'mad',
  'cow',
  'disease',
  'can',
  'be',
  'transmitted',
  'to',
  'sheep',
  '.'],
 ['Germany',
  "'s",
  'representative',
  'to',
  'the',
  'European',
  'Union',
  "'s",
  'veterinary',
  'committee',
  'Werner',
  'Zwingmann',
  'said',
  'on',
  'Wednesday',
  'consumers',
  'should',
  'buy',
  'sheepmeat',
  'from',
  'countries',
  'other',
  'than',
  'Britain',
  'until',
  'the',
  'scientific',
  'advice',
  'was',
  'clearer',
  '.'],
 ['"',
  'We',
  'do',
  "n't",
  'support',
  'any',
  'such',
  'recommendation',
  'because',
  'we',
  'do',
  "n't",
  'see',
  'any',
  'grounds',
  'for',
  'it',
  ',',
  '"',
  'the',
  'Commission',
  "'s",
  'chief',
  'spokesman',
  'Nikolaus',
  'van',
  'der',
  'Pas',
  'told

In [3]:
test = sentences.split()
test 

AttributeError: 'list' object has no attribute 'split'

In [66]:
# Let's create the training, development and testing sets using load_data above
# Remember *not* to include labels in the testing set

dataset = {"train": load_data('eng.train'), "dev": load_data('eng.testa'), "test": load_data('eng.testb',label=False)}

# Fill in the "blank" - replace the "None" above with the correct dataset

# YOUR CODE HERE
#raise NotImplementedError()

In [67]:
# Autograder tests

for subset in dataset:
    assert isinstance(dataset[subset], list), f"Q1: Your {subset} set should be a list. "

for subset, length in zip(dataset.keys(), [7146, 1783, 1516]):
    assert len(dataset[subset]) == length, f"Q1: Your {subset} set is of an incorrect length. "

for subset in dataset:
    for index, sent in enumerate(dataset[subset]):
        assert len(sent) >= 10, f"Q1: Sentence at index {index} of your {subset} set is shorter than required. "
        item_type = tuple if subset in ["train", "dev"] else str
        for item in sent:
            assert isinstance(item, item_type), f"Q1: Your {subset} set should contain lists of {item_type.__name__}s. "

# Some hidden tests below


## Question 2: Train an HMM Tagger (10 pts)

Once we have the training, development and testing sets ready, it's straightforward to train an HMM tagger with the help of the `nltk` library. Specifically, check out the [documentation](http://www.nltk.org/api/nltk.tag.html?#nltk.tag.hmm.HiddenMarkovModelTagger) of the `HiddenMarkovModelTagger` class and understand how to use its classmethod `train` to create an HMM tagger. The classmethod `train` also accepts an argument called `test_sequence`, where you can plug in your development data and get a score as an estimate for how well your HMM will perform on unseen data. 

**This function should return a trained `HiddenMarkovModelTagger` object.**

In [76]:
from nltk.tag.hmm import HiddenMarkovModelTagger

def train_hmm(train, dev):
    """
    Train an HMM tagger on the training set provided
    """
    
    hmm_tagger = HiddenMarkovModelTagger.train(train,dev)
    
    
    
    
    # YOUR CODE HERE
    #raise NotImplementedError()
    
    return hmm_tagger

In [77]:
# Train an HMM tagger, which takes a while

hmm_tagger = train_hmm(dataset["train"], dataset["dev"])

# The number it reports is the accuracy on the development set

accuracy over 43319 tokens: 89.13


In [78]:
# Autograder tests

assert isinstance(hmm_tagger, HiddenMarkovModelTagger), "Q2: Your function should return a HiddenMarkovModelTagger. "
assert hmm_tagger._states, "Q2: Your HMM's states are empty. The tagger wasn't trained properly. "

# Some hidden tests


Now that we have trained an HMM tagger, let's now apply it to your testing set to see how it does. For example, the line of code below tags the first sentence in the testing set. Feel free to adapt the code to tag more sentences. 

In [79]:
hmm_tagger.tag(dataset["test"][0])

[('SOCCER', 'NN'),
 ('-', ':'),
 ('JAPAN', 'NNP'),
 ('GET', '.'),
 ('LUCKY', '"'),
 ('WIN', 'NNP'),
 (',', ','),
 ('CHINA', 'NNP'),
 ('IN', 'IN'),
 ('SURPRISE', 'NNP'),
 ('DEFEAT', 'NNP'),
 ('.', '.')]